# Mastering `StructuredTool.from_function()` in LangChain

## Introduction

In the rapidly evolving landscape of artificial intelligence and natural language processing, the ability to seamlessly integrate custom functionalities into language models is paramount. **StructuredTool.from_function()** emerges as a powerful utility within the LangChain framework, enabling developers to encapsulate Python functions into structured tools that can be effortlessly integrated, executed, and managed within complex AI-driven workflows. This article delves into the versatile applications of `StructuredTool.from_function()`, providing comprehensive examples that illustrate its initialization, execution, batch processing capabilities, error handling mechanisms, and configuration options. Whether you're looking to enhance your AI models with arithmetic operations, text manipulations, or robust error management, this guide offers valuable insights and practical implementations to elevate your development endeavors.

### Comparison Table

To better understand the versatility and applicability of `StructuredTool.from_function()`, the following comparison table highlights the key features and configurations explored in the examples provided:

| **Feature**                        | **Description**                                                                 | **Example(s)**                             |
|------------------------------------|---------------------------------------------------------------------------------|--------------------------------------------|
| **Initialization & Configuration** | Setting up tools with input schemas, metadata, and tags for categorization.      | Creating addition and multiplication tools |
| **Direct Execution**               | Running tools with immediate input and receiving direct outputs.                | Greeting and subtraction tools             |
| **Batch Processing**               | Handling multiple inputs simultaneously with or without additional configurations.| Squaring and cubing numbers                |
| **Error Handling**                 | Managing exceptions gracefully and implementing retry mechanisms.               | Division and random failure tools          |
| **Configuration & Binding**        | Binding fixed arguments and adding configurable fields for dynamic operations.  | Power and repeat tools                     |

This table encapsulates the diverse functionalities that `StructuredTool.from_function()` offers, demonstrating its capability to cater to a wide range of application needs within AI-driven projects.

---

## Preparation

### Installing Required Libraries
This section installs the necessary Python libraries for working with LangChain, OpenAI embeddings, Anthropic models, and other utilities. These libraries include:
- `langchain-openai`: Provides integration with OpenAI's embedding models and APIs.
- `langchain-anthropic`: Enables integration with Anthropic's models and APIs.
- `langchain_community`: Contains community-contributed modules and tools for LangChain.
- `langchain_experimental`: Includes experimental features and utilities for LangChain.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain_community
!pip install -qU langchain_experimental

### Initializing OpenAI and Anthropic Chat Models
This section demonstrates how to securely fetch API keys for OpenAI and Anthropic using Kaggle's `UserSecretsClient` and initialize their respective chat models. The `ChatOpenAI` and `ChatAnthropic` classes are used to create instances of these models, which can be used for natural language processing tasks such as text generation and conversational AI.

**Key steps:**
1. **Fetch API Keys**: The OpenAI and Anthropic API keys are securely retrieved using Kaggle's `UserSecretsClient`.
2. **Initialize Chat Models**:
   - The `ChatOpenAI` class is initialized with the `gpt-4o-mini` model and the fetched OpenAI API key.
   - The `ChatAnthropic` class is initialized with the `claude-3-5-latest` model and the fetched Anthropic API key.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from kaggle_secrets import UserSecretsClient

# Fetch API key securely
user_secrets = UserSecretsClient()

# Initialize LLM
model = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=user_secrets.get_secret("my-openai-api-key"))
#model = ChatAnthropic(model="claude-3-5-latest", temperature=0, api_key=user_secrets.get_secret("my-anthropic-api-key"))

In [ ]:
import json

def pretty_print(aimessage):
    """
    Pretty-prints an AIMessage object by converting it to JSON and formatting it with indentation.
    
    Args:
        aimessage: The AIMessage object to pretty-print.
    """
    # Convert the AIMessage object to a dictionary
    aimessage_dict = {
        "content": aimessage.content,
        "additional_kwargs": aimessage.additional_kwargs,
        "response_metadata": aimessage.response_metadata,
        "id": aimessage.id,
        "tool_calls": aimessage.tool_calls,
        "usage_metadata": aimessage.usage_metadata,
    }
    
    # Convert the dictionary to a JSON-formatted string with indentation
    pretty_json = json.dumps(aimessage_dict, indent=4, ensure_ascii=False)
    
    # Print the pretty JSON
    print(pretty_json)

---

## **1. Initialization and Configuration**

### Example 1: Creating a Tool with Pydantic Schema
This example demonstrates how to initialize and configure a structured tool using `StructuredTool.from_function()`. It showcases defining an input schema with Pydantic's `BaseModel` and `Field`, implementing a simple addition function, creating the tool with the defined schema, executing the tool with specific inputs, and binding the tool to a language model for integration into chains.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Define input schema using Pydantic with Field
class AddInput(BaseModel):
    a: int = Field(description="The first number to add.")
    b: int = Field(description="The second number to add.")

# Define the function
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b

# Create the tool
add_tool = StructuredTool.from_function(
    func=add,
    name="add",
    description="Adds two numbers.",
    args_schema=AddInput,
)

# Use the tool
result = add_tool.run({"a": 3, "b": 5})
print(result)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([add_tool])
ai_msg = model_with_tools.invoke("What is 3 plus 5?")
pretty_print(ai_msg)

---

### Example 2: Creating a Tool with Metadata and Tags
In this example, a structured tool is created with additional metadata and tags to enhance its categorization and discoverability. The process includes defining an input schema with Pydantic, implementing a multiplication function, and utilizing `StructuredTool.from_function()` with metadata and tags. The tool is then executed with specific inputs and bound to a language model for use within chains.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Define input schema with Field
class MultiplyInput(BaseModel):
    x: int = Field(description="The first number to multiply.")
    y: int = Field(description="The second number to multiply.")

# Define the function
def multiply(x: int, y: int) -> int:
    """Multiply two numbers."""
    return x * y

# Create the tool with metadata and tags
multiply_tool = StructuredTool.from_function(
    func=multiply,
    name="multiply",
    description="Multiplies two numbers.",
    args_schema=MultiplyInput,
    metadata={"category": "math"},
    tags=["arithmetic", "math"],
)

# Use the tool
result = multiply_tool.run({"x": 4, "y": 6})
print(result)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([multiply_tool])
ai_msg = model_with_tools.invoke("What is 4 multiplied by 6?")
pretty_print(ai_msg)

---

## **2. Execution and Running**

### Example 3: Running a Tool with Direct Input
This example illustrates how to execute a structured tool by providing direct input. It involves defining an input schema with Pydantic, implementing a greeting function, creating the tool using `StructuredTool.from_function()`, running the tool with specific input data, and binding the tool to a language model for integration into conversational chains.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Define input schema with Field
class GreetInput(BaseModel):
    name: str = Field(description="The name of the person to greet.")

# Define the function
def greet(name: str) -> str:
    """Greet a person."""
    return f"Hello, {name}!"

# Create the tool
greet_tool = StructuredTool.from_function(
    func=greet,
    name="greet",
    description="Greets a person by name.",
    args_schema=GreetInput,
)

# Use the tool
result = greet_tool.run({"name": "Alice"})
print(result)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([greet_tool])
ai_msg = model_with_tools.invoke("Greet Alice.")
pretty_print(ai_msg)

### Example 4: Running a Tool with Return Direct
This example showcases how to execute a tool with the `return_direct` parameter set to `True`, allowing the tool to return results directly without additional processing. It includes defining an input schema, implementing a subtraction function, creating the tool with `return_direct=True`, running the tool with specific inputs, and binding the tool to a language model for use in chains.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Define input schema with Field
class SubtractInput(BaseModel):
    a: int = Field(description="The number to subtract from.")
    b: int = Field(description="The number to subtract.")

# Define the function
def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b

# Create the tool with return_direct=True
subtract_tool = StructuredTool.from_function(
    func=subtract,
    name="subtract",
    description="Subtracts two numbers.",
    args_schema=SubtractInput,
    return_direct=True,
)

# Use the tool
result = subtract_tool.run({"a": 10, "b": 4})
print(result)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([subtract_tool])
ai_msg = model_with_tools.invoke("What is 10 minus 4?")
pretty_print(ai_msg)

---

## **3. Batch Processing**

### Example 5: Batch Processing with Multiple Inputs
This example demonstrates how to perform batch processing using a structured tool. It involves defining an input schema for squaring numbers, implementing the square function, creating the tool, preparing multiple inputs, executing the batch processing, and binding the tool to a language model for handling multiple requests in a single operation.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Step 1: Define the input schema using Pydantic with Field
# This schema describes the expected input for the tool.
class SquareInput(BaseModel):
    number: int = Field(description="The number to square.")

# Step 2: Define the function that the tool will execute
# This function calculates the square of a number.
def square(number: int) -> int:
    """Square a number."""
    return number**2

# Step 3: Create the tool using StructuredTool.from_function
# This wraps the `square` function into a reusable tool with input validation.
square_tool = StructuredTool.from_function(
    func=square,  # The function to wrap
    name="square",  # Name of the tool
    description="Squares a number.",  # Tool description
    args_schema=SquareInput,  # Input schema for validation
)

# Step 4: Prepare a list of inputs for batch processing
# Each input is a dictionary that matches the tool's input schema.
inputs = [
    {"number": 2},  # Input 1: Square of 2
    {"number": 3},  # Input 2: Square of 3
    {"number": 4},  # Input 3: Square of 4
]

# Step 5: Perform batch processing
# The `batch` method processes multiple inputs in parallel.
# It returns a list of results corresponding to each input.
results = square_tool.batch(inputs)

# Step 6: Print the results
# The output is a list of squared values for each input.
print(results)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([square_tool])
ai_msg = model_with_tools.invoke("Square the numbers 2, 3, and 4.")
pretty_print(ai_msg)

### Example 6: Batch Processing with Config
This example illustrates batch processing with additional configuration parameters. It defines an input schema for cubing numbers, implements the cube function, creates the tool, prepares multiple inputs, and executes batch processing with a configuration that limits the maximum concurrency of parallel executions. The tool is then bound to a language model for efficient handling of multiple requests with controlled resource usage.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Step 1: Define the input schema using Pydantic with Field
# This schema describes the expected input for the tool.
class CubeInput(BaseModel):
    number: int = Field(description="The number to cube.")

# Step 2: Define the function that the tool will execute
# This function calculates the cube of a number.
def cube(number: int) -> int:
    """Cube a number."""
    return number**3

# Step 3: Create the tool using StructuredTool.from_function
# This wraps the `cube` function into a reusable tool with input validation.
cube_tool = StructuredTool.from_function(
    func=cube,  # The function to wrap
    name="cube",  # Name of the tool
    description="Cubes a number.",  # Tool description
    args_schema=CubeInput,  # Input schema for validation
)

# Step 4: Prepare a list of inputs for batch processing
# Each input is a dictionary that matches the tool's input schema.
inputs = [{"number": 2}, {"number": 3}]

# Step 5: Perform batch processing with a configuration
# The `batch` method processes multiple inputs in parallel.
# The `config` parameter allows you to control the behavior of the batch process.
# Here, we set `max_concurrency=2` to limit the number of parallel executions to 2.
results = cube_tool.batch(inputs, config={"max_concurrency": 2})

# Step 6: Print the results
# The output is a list of results corresponding to each input.
print(results)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([cube_tool])
ai_msg = model_with_tools.invoke("Cube the numbers 2 and 3.")
pretty_print(ai_msg)

## **4. Error Handling and Retries**

### Example 7: Handling Tool Errors
This example focuses on error handling within a structured tool. It defines an input schema for division operations, implements a division function that raises an error when attempting to divide by zero, and creates the tool with error handling enabled. The tool is then executed with invalid input to demonstrate how errors are managed, and it is bound to a language model for robust error-aware interactions within chains.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Define input schema with Field
class DivideInput(BaseModel):
    a: int = Field(description="The dividend.")
    b: int = Field(description="The divisor (must not be zero).")

# Define the function
def divide(a: int, b: int) -> float:
    """Divide two numbers."""
    if b == 0:
        raise ValueError("Cannot divide by zero.")
    return a / b

# Create the tool with error handling
divide_tool = StructuredTool.from_function(
    func=divide,
    name="divide",
    description="Divides two numbers.",
    args_schema=DivideInput,
    handle_tool_error=True,  # Enable error handling
)

# Use the tool
try:
    result = divide_tool.run({"a": 10, "b": 0})
    print(result)
except Exception as e:
    print(f"Tool error: {e}")

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([divide_tool])
ai_msg = model_with_tools.invoke("Divide 10 by 0.")
pretty_print(ai_msg)

### Example 8: Retrying on Exceptions
This example demonstrates how to create a `StructuredTool` that wraps a function prone to random failures and adds retry logic to handle such failures gracefully. The following key points are covered:

1. **TypedDict Schema:** The input schema is defined using `TypedDict` and `Annotated` to describe the expected parameters and their purposes clearly.
2. **Random Failure Function:** The `random_fail` function simulates random failures based on a specified failure probability (`failure_rate`) and returns a success message if it doesn't fail.
3. **Retry Mechanism:** The `with_retry` method is used to configure the tool to retry up to 3 times if the function fails due to an exception.
4. **Example Usage:** Two scenarios are tested:
   - A high failure rate (80%) to demonstrate retries and error handling.
   - A lower failure rate (30%) to show a successful execution after retries.

In [ ]:
from langchain.tools import StructuredTool
from typing import TypedDict
from typing_extensions import Annotated
import random

# Define the input schema using TypedDict
class RandomFailInput(TypedDict):
    """Input for random fail function."""
    failure_rate: Annotated[float, "Probability of failure between 0 and 1"]
    message: Annotated[str, "Message to return on success"]

# Define the function that will randomly fail
def random_fail(failure_rate: float, message: str) -> str:
    """
    A function that randomly fails based on the given failure rate.
    
    Args:
        failure_rate: Probability of failure (between 0 and 1)
        message: Message to return on success
    """
    if random.random() < failure_rate:
        raise ValueError("Random failure occurred!")
    return f"Success: {message}"

# Create the tool with retries
random_fail_tool = StructuredTool.from_function(
    func=random_fail,
    name="random_fail",
    description="Randomly fails for demonstration.",
).with_retry(stop_after_attempt=3)  # Retry up to 3 times

# Example usage
try:
    # High failure rate to demonstrate retries
    result = random_fail_tool.invoke(input={
        "failure_rate": 0.8,  # 80% chance of failure
        "message": "Operation completed!"
    })
    print(f"Final result: {result}")
except Exception as e:
    print(f"All retries failed. Final error: {str(e)}")

In [ ]:
# Example with lower failure rate
try:
    # Lower failure rate more likely to succeed
    result = random_fail_tool.invoke(input={
        "failure_rate": 0.3,  # 30% chance of failure
        "message": "This attempt should work!"
    })
    print(f"Final result: {result}")
except Exception as e:
    print(f"All retries failed. Final error: {str(e)}")

---

## **5. Configuration and Binding**

### Example 9: Binding Arguments
This example illustrates how to bind specific arguments to a structured tool, effectively pre-configuring certain parameters. It involves defining an input schema for exponentiation, implementing a power function, creating the tool, binding the base argument to a fixed value, and executing the bound tool with varying exponents. The tool is then bound to a language model for seamless integration into chains with pre-configured parameters.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Step 1: Define the input schema using Pydantic with Field
# This schema describes the expected input for the tool.
class PowerInput(BaseModel):
    base: int = Field(description="The base number.")
    exponent: int = Field(description="The exponent to raise the base to.")

# Step 2: Define the function that the tool will execute
# This function calculates the power of a base raised to an exponent.
def power(base: int, exponent: int) -> int:
    """Raise base to the power of exponent."""
    return base**exponent

# Step 3: Create the tool using StructuredTool.from_function
# This wraps the `power` function into a reusable tool with input validation.
power_tool = StructuredTool.from_function(
    func=power,  # The function to wrap
    name="power",  # Name of the tool
    description="Raises base to the power of exponent.",  # Tool description
    args_schema=PowerInput,  # Input schema for validation
)

# Step 4: Bind arguments to the tool
# The `bind` method allows you to fix certain arguments of the tool.
# Here, we bind `base=2`, so the tool only requires `exponent` as input.
bound_tool = power_tool.bind(base=2)

# Step 5: Use the bound tool
# Now, the tool only needs the `exponent` input since `base` is already fixed.
# We need to modify the input to include the bound `base` value.
result = bound_tool.run({"base": 2, "exponent": 3})  # Include `base` in the input
print("Step-5:", result)  # Output: 8

# Step 6: Reuse the bound tool with a different exponent
# This demonstrates how the same bound tool can be reused with different inputs.
result = bound_tool.run({"base": 2, "exponent": 4})  # Include `base` in the input
print("Step-6:", result)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([power_tool])
ai_msg = model_with_tools.invoke("Calculate 2 to the power of 3.")
pretty_print(ai_msg)

### Example 10: Configurable Fields
This example demonstrates how to use `ConfigurableField` to dynamically adjust the behavior of a language model, specifically its temperature parameter, which controls the randomness and creativity of its outputs. 

1. **Configurable Field Definition:**
   - A `ConfigurableField` named `model_temperature` is defined, providing a clear description of how temperature influences the model's output. Lower values make the output more deterministic, while higher values make it more creative.
2. **ChatOpenAI Model Initialization:**
   - A `ChatOpenAI` model is created with a default temperature of `0.7`. The temperature field is made configurable, allowing adjustments at runtime.
3. **Using the Default Temperature (0.7):**
   - The model generates output with the default temperature, which balances creativity and determinism.
4. **Reconfiguring to a Lower Temperature (0.2):**
   - The temperature is set to a lower value, resulting in more deterministic and less random output.
5. **Reconfiguring to a Higher Temperature (1.0):**
   - The temperature is set to a higher value, encouraging more creative and varied responses.

In [ ]:
from langchain_core.runnables import ConfigurableField
from langchain_openai import ChatOpenAI

config = ConfigurableField(
        id="model_temperature",  # Unique identifier for the configurable field
        name="Model Temperature",  # Human-readable name for the field
        description="Controls the randomness of the model's output. Lower values make the output more deterministic, while higher values make it more creative.",  # Description of the field
    )

# Step 1: Create a ChatOpenAI model with a configurable temperature
llm = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0.7, 
                 api_key=user_secrets.get_secret("my-openai-api-key")).configurable_fields(temperature=config)

# Step 2: Use the model with the default temperature (0.7)
print(
    "Temperature 0.7 (Default): ",
    llm.invoke("Tell me a joke.").content
)

In [ ]:
# Step 3: Reconfigure the model with a lower temperature (0.2)
# Lower temperature makes the output more deterministic.
print(
    "Temperature 0.2: ",
    llm.with_config(configurable={"model_temperature": 0.2}).invoke("Tell me a joke.").content
)

In [ ]:
# Step 4: Reconfigure the model with a higher temperature (1.0)
# Higher temperature makes the output more creative.
print(
    "Temperature 1.0: ",
    llm.with_config(configurable={"model_temperature": 1.0}).invoke("Tell me a joke.").content
)

---

## **Best Practices**

### Example 11: Using All Parameters of `from_function`
This example demonstrates how to fully utilize all the parameters of the `from_function` method in LangChain's `StructuredTool`. The example highlights advanced customization options such as:

1. **Custom Pydantic Schema:** A defined schema ensures precise validation for the inputs.
2. **Google-Style Docstring Parsing:** The function's docstring is parsed to infer schema and enhance the tool's documentation.
3. **Response Formatting:** The tool outputs a tuple consisting of a message and metadata.
4. **Error Handling:** Errors are raised if the docstring format is invalid.
5. **Metadata and Tags:** Metadata is added for categorization, and tags help describe the tool's purpose.
6. **Direct Response Return:** The result is returned directly without additional processing.

This advanced setup is ideal for complex functions that require detailed customization and rigorous validation.

In [ ]:
from pydantic import BaseModel, Field
from typing import Awaitable, Any, Literal
from langchain_core.tools import StructuredTool

# Define a function with a Google-style docstring
def complex_function(a: int, b: str) -> tuple[str, dict]:
    """
    A complex function that processes inputs and returns a tuple.

    Args:
        a (int): The first input, an integer.
        b (str): The second input, a string.

    Returns:
        tuple[str, dict]: A tuple containing a message and a metadata dictionary.
    """
    message = f"Processed: {a} and {b}"
    metadata = {"input_a": a, "input_b": b}
    return message, metadata

# Define a custom Pydantic schema for input validation
class ComplexInput(BaseModel):
    a: int = Field(description="The first input, an integer.")
    b: str = Field(description="The second input, a string.")

# Create the tool using all parameters of from_function
complex_tool = StructuredTool.from_function(
    func=complex_function,  # The function to wrap
    coroutine=None,  # No async version provided
    name="complex_tool",  # Custom name
    description="A tool that processes inputs and returns a tuple.",  # Custom description
    return_direct=True,  # Return the result directly
    args_schema=ComplexInput,  # Use the custom Pydantic schema
    infer_schema=True,  # Infer schema from the function signature
    response_format="content_and_artifact",  # Expect a tuple as output
    parse_docstring=True,  # Parse the Google-style docstring
    error_on_invalid_docstring=True,  # Raise an error if the docstring is invalid
    metadata={"category": "demo"},  # Additional metadata
    tags=["example", "advanced"],  # Tags for categorization
)

# Use the tool
result = complex_tool.run({"a": 42, "b": "example"})
print(result)

In [ ]:
# Bind the tool to the LLM for use in chains
model_with_tools = model.bind_tools([complex_tool])
ai_msg = model_with_tools.invoke("Repeat 'hi' 3 times.")
pretty_print(ai_msg)

### Example 12: Using Multiple Tools with `from_function`
This example demonstrates how to use `ConfigurableField` to dynamically adjust the behavior of a language model, specifically its temperature parameter, which controls the randomness and creativity of its outputs. 

1. **Configurable Field Definition:**
   - A `ConfigurableField` named `model_temperature` is defined, providing a clear description of how temperature influences the model's output. Lower values make the output more deterministic, while higher values make it more creative.
2. **ChatOpenAI Model Initialization:**
   - A `ChatOpenAI` model is created with a default temperature of `0.7`. The temperature field is made configurable, allowing adjustments at runtime.
3. **Using the Default Temperature (0.7):**
   - The model generates output with the default temperature, which balances creativity and determinism.
4. **Reconfiguring to a Lower Temperature (0.2):**
   - The temperature is set to a lower value, resulting in more deterministic and less random output.
5. **Reconfiguring to a Higher Temperature (1.0):**
   - The temperature is set to a higher value, encouraging more creative and varied responses.

This setup is particularly useful for applications requiring dynamic control over the model's behavior, such as interactive systems or scenarios that demand varied levels of creativity.

In [ ]:
from pydantic import BaseModel, Field
from langchain_core.tools import StructuredTool

# Step 1: Define input schemas using Pydantic with Field

# Schema for the "currency_converter" tool
class CurrencyConverterInput(BaseModel):
    amount: float = Field(description="The amount of money to convert.")
    from_currency: str = Field(description="The currency to convert from (e.g., USD).")
    to_currency: str = Field(description="The currency to convert to (e.g., EUR).")

# Schema for the "weather_lookup" tool
class WeatherLookupInput(BaseModel):
    location: str = Field(description="The location to check the weather for (e.g., New York).")

# Schema for the "text_summarizer" tool
class TextSummarizerInput(BaseModel):
    text: str = Field(description="The text to summarize.")
    max_length: int = Field(default=100, description="The maximum length of the summary.")

# Step 2: Define the functions for each tool

# Function for the "currency_converter" tool
def currency_converter(amount: float, from_currency: str, to_currency: str) -> str:
    """Convert an amount from one currency to another."""
    # Simulate a currency conversion (in a real-world scenario, this would call an API)
    conversion_rates = {
        "USD": {"EUR": 0.93, "GBP": 0.80, "JPY": 148.50},
        "EUR": {"USD": 1.07, "GBP": 0.86, "JPY": 159.50},
        "GBP": {"USD": 1.25, "EUR": 1.16, "JPY": 185.00},
        "JPY": {"USD": 0.0067, "EUR": 0.0063, "GBP": 0.0054},
    }
    if from_currency not in conversion_rates or to_currency not in conversion_rates[from_currency]:
        return f"Conversion from {from_currency} to {to_currency} is not supported."
    rate = conversion_rates[from_currency][to_currency]
    converted_amount = amount * rate
    return f"{amount} {from_currency} = {converted_amount:.2f} {to_currency}"

# Function for the "weather_lookup" tool
def weather_lookup(location: str) -> str:
    """Get the current weather for a given location."""
    # Simulate a weather lookup (in a real-world scenario, this would call an API)
    weather_data = {
        "New York": "Sunny, 72°F",
        "London": "Cloudy, 55°F",
        "Tokyo": "Rainy, 65°F",
        "Paris": "Partly cloudy, 68°F",
    }
    if location not in weather_data:
        return f"Weather data for {location} is not available."
    return f"The weather in {location} is {weather_data[location]}."

# Function for the "text_summarizer" tool
def text_summarizer(text: str, max_length: int = 100) -> str:
    """Summarize a given text to a specified maximum length."""
    # Simulate text summarization (in a real-world scenario, this would use an NLP model)
    if len(text) <= max_length:
        return text
    return text[:max_length] + "..."

# Step 3: Create the tools using StructuredTool.from_function

# Create the "currency_converter" tool
currency_converter_tool = StructuredTool.from_function(
    func=currency_converter,
    name="currency_converter",
    description="Converts an amount from one currency to another.",
    args_schema=CurrencyConverterInput,
)

# Create the "weather_lookup" tool
weather_lookup_tool = StructuredTool.from_function(
    func=weather_lookup,
    name="weather_lookup",
    description="Gets the current weather for a given location.",
    args_schema=WeatherLookupInput,
)

# Create the "text_summarizer" tool
text_summarizer_tool = StructuredTool.from_function(
    func=text_summarizer,
    name="text_summarizer",
    description="Summarizes a given text to a specified maximum length.",
    args_schema=TextSummarizerInput,
)

# Step 4: Use the tools

# Use the "currency_converter" tool
conversion_result = currency_converter_tool.run({"amount": 100, "from_currency": "USD", "to_currency": "EUR"})
print("Currency Conversion Result:", conversion_result)  # Output: 100 USD = 93.00 EUR

# Use the "weather_lookup" tool
weather_result = weather_lookup_tool.run({"location": "New York"})
print("Weather Lookup Result:", weather_result)  # Output: The weather in New York is Sunny, 72°F

# Use the "text_summarizer" tool
text = "The quick brown fox jumps over the lazy dog. This is a long text that needs to be summarized."
summary_result = text_summarizer_tool.run({"text": text, "max_length": 20})
print("Text Summarizer Result:", summary_result)  # Output: The quick brown fox...

# Step 5: Bind the tools to the LLM for use in chains
model_with_tools = model.bind_tools([currency_converter_tool, weather_lookup_tool, text_summarizer_tool])

In [ ]:
# Use the model with the bound tools
ai_msg = model_with_tools.invoke("Convert 100 USD to EUR and check the weather in London.")
pretty_print(ai_msg)

In [ ]:
# Use the model with the bound tools
ai_msg = model_with_tools.invoke("Convert 200 USD to JPY.")
pretty_print(ai_msg)

In [ ]:
# Use the model with the bound tools
ai_msg = model_with_tools.invoke("What is the weather like in Tokyo?")
pretty_print(ai_msg)

In [ ]:
# Use the model with the bound tools
ai_msg = model_with_tools.invoke("Summarize this text: 'The quick brown fox jumps over the lazy dog. This is a long text that needs to be summarized.'")
pretty_print(ai_msg)

In [ ]:
# Use the model with the bound tools
ai_msg = model_with_tools.invoke("Tell me a joke.")
pretty_print(ai_msg)

## Conclusion

`StructuredTool.from_function()` serves as a cornerstone for developers aiming to extend the capabilities of language models with custom, structured functionalities. Through the comprehensive examples provided, we've explored how this utility facilitates the creation of robust tools encompassing initialization with detailed schemas, direct and batch executions, sophisticated error handling, and dynamic configurations. By leveraging Pydantic for input validation and integrating metadata and tags for enhanced tool management, developers can craft precise and reliable tools tailored to specific tasks. Furthermore, the ability to bind arguments and configure fields at runtime adds a layer of flexibility, making `StructuredTool.from_function()` an indispensable asset in building scalable and maintainable AI applications. As the demand for more interactive and intelligent systems grows, mastering tools like `StructuredTool.from_function()` will be pivotal in driving innovation and efficiency in AI development.